In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [8]:
spark = SparkSession.builder.appName("Date_Dim").getOrCreate()

In [9]:
hdfs_bronze_path ="hdfs:///data/bronze/Olympics_Games_parquet"
df = spark.read.parquet(hdfs_bronze_path)

In [10]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [11]:
df.select("year","start_date","end_date").show(5)

+----+----------+--------+
|year|start_date|end_date|
+----+----------+--------+
|1896|   6 April|15 April|
|1900|      NULL|    NULL|
|1904|    14 May|    NULL|
|1908|   13 July| 25 July|
|1912|    6 July| 15 July|
+----+----------+--------+
only showing top 5 rows



In [12]:
df.select("year","start_date","end_date").printSchema()

root
 |-- year: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)



In [13]:
df_New = df.select(col("edition_id").alias("Date_id").cast("integer"),col("year"),col("start_date"),col("end_date"))

In [ ]:
df_New = df_New.withColumn("year", col("year").cast("int"))

df_New = df_New.withColumn("start_date", to_date(trim(col("start_date")), "d MMMM"))
df_New = df_New.withColumn("end_date", to_date(trim(col("end_date")), "d MMMM"))



In [15]:
df_New = df_New.withColumn("start_day_date", dayofmonth(col("start_date")))
df_New = df_New.withColumn("start_month_date", month(col("start_date")))
df_New = df_New.withColumn("end_day_date", dayofmonth(col("end_date")))
df_New = df_New.withColumn("end_month_date", month(col("end_date")))

In [16]:
df_New = df_New.select("Date_id","year","start_day_date","start_month_date","end_day_date","end_month_date")

In [17]:
df.select("year","start_date","end_date").show(5)

+----+----------+--------+
|year|start_date|end_date|
+----+----------+--------+
|1896|   6 April|15 April|
|1900|      NULL|    NULL|
|1904|    14 May|    NULL|
|1908|   13 July| 25 July|
|1912|    6 July| 15 July|
+----+----------+--------+
only showing top 5 rows



In [21]:
df_New.show(10)

+-------+----+--------------+----------------+------------+--------------+
|Date_id|year|start_day_date|start_month_date|end_day_date|end_month_date|
+-------+----+--------------+----------------+------------+--------------+
|      1|1896|             6|               4|          15|             4|
|      2|1900|          NULL|            NULL|        NULL|          NULL|
|      3|1904|            14|               5|        NULL|          NULL|
|      5|1908|            13|               7|          25|             7|
|      6|1912|             6|               7|          15|             7|
|     50|1916|          NULL|            NULL|        NULL|          NULL|
|      7|1920|            14|               8|          30|             8|
|      8|1924|             5|               7|          27|             7|
|      9|1928|            28|               7|          12|             8|
|     10|1932|            30|               7|          14|             8|
+-------+----+-----------

In [19]:
df.printSchema()

root
 |-- edition: string (nullable = true)
 |-- edition_id: string (nullable = true)
 |-- edition_url: string (nullable = true)
 |-- year: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country_flag_url: string (nullable = true)
 |-- country_noc: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- competition_date: string (nullable = true)
 |-- isHeld: string (nullable = true)



In [14]:
df_New.count()

64

In [ ]:
df_New.write.mode("overwrite").parquet("hdfs:///data/silver/Date_parquet")